In [16]:
import time
import akshare as ak
import pandas as pd
import numpy as np
import tqdm
# import pyecharts.options as opts
# from pyecharts.charts import Line
import pandas as pd
from sqlalchemy import create_engine
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

### 获取个股信息

In [44]:
df=ak.stock_zh_a_hist(
        symbol="000933",
        period="daily",
        start_date='20250101',
        # end_date=end_date,
        adjust="",
    )

### 抽取最近一个月数据，分析下趋势线

In [45]:
df=df.sort_values('日期',ascending=False).iloc[0:30,:]
df=df.sort_values('日期')
df['t']=np.arange(len(df))

In [46]:
df.head()

,日期,股票代码,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率,t
181,2025-09-29,000933,19.50,19.78,19.81,19.09,497726,9.717307e+08,3.72,2.17,0.42,2.21,0
182,2025-09-30,000933,19.71,20.01,20.38,19.62,404858,8.113761e+08,3.84,1.16,0.23,1.80,1
183,2025-10-09,000933,20.68,21.21,21.24,20.08,790861,1.637621e+09,5.80,6.00,1.20,3.52,2
184,2025-10-10,000933,20.99,21.65,22.01,20.77,771479,1.664364e+09,5.85,2.07,0.44,3.43,3
185,2025-10-13,000933,20.80,21.28,21.39,20.59,657934,1.380835e+09,3.70,-1.71,-0.37,2.93,4


In [47]:
model = LinearRegression()
model.fit(df[['t']], df['最低'])

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [48]:
slope = model.coef_[0]
intercept = model.intercept_

print("趋势线斜率 =", slope)
print("趋势线截距 =", intercept)

# 计算整段趋势线
df['TrendLine'] = model.predict(df[['t']])

趋势线斜率 = 0.24107452725250278
趋势线截距 = 19.930086021505375


In [ ]:
from pyecharts.charts import Kline, Line, Grid
from pyecharts import options as opts
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

def kline_with_low_regression(df):
    # --- 1. 准备数据 ---
    df = df.copy()
    df['t'] = np.arange(len(df))

    # 选最低价中最低的 20% 用于拟合趋势线（更接近人工画法）
    # threshold = df['Low'].quantile(0.2)
    # df_fit = df[df['Low'] <= threshold]

    # --- 2. 线性回归 ---
    model = LinearRegression()
    model.fit(df[['t']], df['最低'],sample_weight=df['成交量'])
    df['TrendLine'] = model.predict(df[['t']])

    # --- 3. 构造 K 线图 ---
    kline = (
        Kline()
        .add_xaxis(df['日期'].tolist())
        .add_yaxis(
            "K线",
            df[['开盘', '收盘', '最低', '最高']].values.tolist(),
            itemstyle_opts=opts.ItemStyleOpts(color="#ec0000", color0="#00da3c",
                                              border_color="#8A0000", border_color0="#008F28"),
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title="K线 + Low 回归趋势线"),
            datazoom_opts=[opts.DataZoomOpts(), opts.DataZoomOpts(type_="inside")],
            xaxis_opts=opts.AxisOpts(type_="category"),
            # yaxis_opts=opts.AxisOpts(scale=True),
            tooltip_opts=opts.TooltipOpts(trigger="axis")
        )
    )

    # --- 4. 趋势线 ---
    line = (
        Line()
        .add_xaxis(df['日期'].tolist())
        .add_yaxis(
            "Low 回归趋势线",
            df['TrendLine'].round(2).tolist(),
            is_smooth=True,
            linestyle_opts=opts.LineStyleOpts(width=2, color="orange"),
            label_opts=opts.LabelOpts(is_show=False),
        )
    )

    # --- 5. 叠加图层 ---
    overlap = kline.overlap(line)

    # --- 6. 使用 Grid 合成图 ---
    grid = (
        Grid()
        .add(overlap, grid_opts=opts.GridOpts())
    )
    return grid
# 假设 df 已有你的行情数据
chart = kline_with_low_regression(df)
chart.render("kline_regression.html")
# chart.render_notebook()


'c:\\Users\\cyw\\Desktop\\jupyternotebook\\git-python\\GP\\趋势分析\\kline_regression.html'

In [50]:
# 假设 df 已有你的行情数据
# chart = kline_with_low_regression(df)
# chart.render("kline_regression.html")
chart.render_notebook()
